In [17]:
from prep_data import Data_prepare
import numpy as np 
import os 
# import prep_data

###  Data preprocessing 
1. covert mat data to numpy array 
2. combine male and female data to one array 
3. if needed, remove certain rows 
4. get postive and negative of Xs 
    - order difference or net vs neg 
5. reshape each X: 128*8*2 = 2048 
    - input: data 
6. generate labels for x 
    - input: data and label_type (0:zeros, 1: ones )
7. combine net and neg 
    - input: netX,netY,negX,negY

In [2]:
dp = Data_prepare() 
pos = [[0,3],[0,2],[0,1],[0,0]]
neg = [[1,3],[1,2],[1,1],[1,0]]
# netural vs postive 
# pos = [[0,0],[0,1],[1,2],[1,3],[2,0],[2,1],[2,2],[2,3]]
# neg = [[0,2],[0,3],[1,0],[1,1],[3,0],[3,1],[3,2],[3,3]] 
m_file = "./raw_data/Emotrans1_Boy_data_preprocessed_42.mat" 
f_file = "./raw_data/Emotrans1_girl_data_preprocessed_42.mat"
index=[40,41]
df = dp.combine_male_female(m_file,f_file)
df = dp.remove_person(df,index)
pos_df = dp.get_pos_or_neg(df,pos)
neg_df = dp.get_pos_or_neg(df,neg)
pos_sequeezed = dp.squeeze_feature_size(pos_df)
neg_sequeezed = dp.squeeze_feature_size(neg_df)
pos_labels = dp.generate_labels(pos_sequeezed,1)
neg_labels = dp.generate_labels(neg_sequeezed,0)
x,y = dp.combine_net_neg(pos_sequeezed,pos_labels,neg_sequeezed,neg_labels)

/Users/allen/Documents/course/4490/Jan.29/allen/prep_data.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(data_dict_female["All_Feature"])
/Users/allen/Documents/course/4490/Jan.29/allen/prep_data.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(data_dict_female["All_Feature"])
/Users/allen/Documents/course/4490/Jan.29/allen/prep_data.py:51: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is dep

In [3]:
## (40,) -> 40*36*2048
def flatten(data):
    res = data[0]
    for i in range(1,data.shape[0]):
        res = np.append(res,data[i],axis=0)
    return res 
new_x = flatten(x)
new_y = flatten(y)

In [4]:
print(new_x.shape,new_y.shape)


(1404, 2048) (1404, 1)


#### Normalize x with standardScaler

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(new_x)
new_x = scaler.transform(new_x)

In [6]:
print(new_x.shape,new_y.shape)

(1404, 2048) (1404, 1)


In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(new_x, new_y, test_size = 0.1, random_state = 4)

In [8]:
print(x_train.shape,y_train.shape)

(1263, 2048) (1263, 1)


In [9]:
## reshape x_train and x_test to 3D for LSTM 
x_train = np.reshape(x_train, (x_train.shape[0],1,new_x.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0],1,new_x.shape[1]))

In [10]:
# y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
# y_test = np.asarray(y_test).astype('float32').reshape((-1,1))

In [11]:
print(x_train.shape,x_test.shape)
print(y_train.shape,y_test.shape)

(1263, 1, 2048) (141, 1, 2048)
(1263, 1) (141, 1)


#### LSTM model 

In [18]:
import tensorflow as tf
from tensorflow.keras import Sequential

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
tf.keras.backend.clear_session()

model = Sequential()
model.add(LSTM(128, input_shape=(1,2048),activation="relu",return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32,activation="sigmoid",return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(16,activation="sigmoid"))
model.add(Dropout(0.2))
# model.add(LSTM(50))
# model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))
from keras.optimizers import SGD
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 128)            1114624   
_________________________________________________________________
dropout (Dropout)            (None, 1, 128)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 1, 32)             20608     
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 32)             0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 16)                3136      
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 2)                 3

In [21]:
import datetime
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
history = model.fit(x_train, y_train, epochs = 100, validation_data= (x_test, y_test),callbacks=[tensorboard_callback])
score, acc = model.evaluate(x_test, y_test)

Epoch 1/100
40/40 [==============================] - 3s 23ms/step - loss: 0.6879 - accuracy: 0.5440 - val_loss: 0.6758 - val_accuracy: 0.6312
Epoch 2/100
40/40 [==============================] - 0s 9ms/step - loss: 0.6723 - accuracy: 0.5925 - val_loss: 0.6443 - val_accuracy: 0.6809
Epoch 3/100
40/40 [==============================] - 0s 9ms/step - loss: 0.6362 - accuracy: 0.6892 - val_loss: 0.6219 - val_accuracy: 0.6879
Epoch 4/100
40/40 [==============================] - 0s 9ms/step - loss: 0.5987 - accuracy: 0.7096 - val_loss: 0.6017 - val_accuracy: 0.6950
Epoch 5/100
40/40 [==============================] - 0s 10ms/step - loss: 0.5798 - accuracy: 0.7289 - val_loss: 0.6148 - val_accuracy: 0.6809
Epoch 6/100
40/40 [==============================] - 0s 9ms/step - loss: 0.5786 - accuracy: 0.7296 - val_loss: 0.5833 - val_accuracy: 0.6950
Epoch 7/100
40/40 [==============================] - 0s 9ms/step - loss: 0.5647 - accuracy: 0.7275 - val_loss: 0.5906 - val_accuracy: 0.7092
Epoch 8/100

Epoch 59/100
40/40 [==============================] - 0s 9ms/step - loss: 0.3191 - accuracy: 0.8944 - val_loss: 0.6894 - val_accuracy: 0.7305
Epoch 60/100
40/40 [==============================] - 0s 9ms/step - loss: 0.3126 - accuracy: 0.9098 - val_loss: 0.6930 - val_accuracy: 0.7376
Epoch 61/100
40/40 [==============================] - 0s 9ms/step - loss: 0.2881 - accuracy: 0.9098 - val_loss: 0.7092 - val_accuracy: 0.7163
Epoch 62/100
40/40 [==============================] - 0s 9ms/step - loss: 0.2922 - accuracy: 0.9021 - val_loss: 0.6583 - val_accuracy: 0.7447
Epoch 63/100
40/40 [==============================] - 0s 9ms/step - loss: 0.2889 - accuracy: 0.9102 - val_loss: 0.6714 - val_accuracy: 0.7234
Epoch 64/100
40/40 [==============================] - 0s 9ms/step - loss: 0.2757 - accuracy: 0.9134 - val_loss: 0.7006 - val_accuracy: 0.7234
Epoch 65/100
40/40 [==============================] - 0s 9ms/step - loss: 0.2751 - accuracy: 0.9104 - val_loss: 0.7050 - val_accuracy: 0.7376
Epoch 

In [22]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [23]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 14017), started 0:03:03 ago. (Use '!kill 14017' to kill it.)